## Testin Zone:
Testing classes used in the agent

In [152]:
from kaggle_environments.envs.halite.helpers import *
import pandas as pd
    
class ShipLocation:
    def __init__(self, board, ship):
        self.board = board
        self.ship = ship
        # Get the grid
        self.grid = grid_5(ship.cell)
        
    def get_ship_info(self):
        """ Returns the info about ships in all of the board. """
        ships_info = {}
        
        for ship_id, ship in self.board.ships.items():
            base_info = {
                "my_ship": 0, 
                "cargo": 0, "moves": 0
            }
            
            base_info['cargo'] = ship.cell.halite
            base_info['moves'] = count_moves(self.ship.position, ship.position)
            
            if ship_id in self.board.current_player.ship_ids and ship.id != self.ship.id:
                base_info['my_ship'] = 1
                ships_info[ship_id] = base_info
            elif not(ship_id in self.board.current_player.ship_ids):
                ships_info[ship_id] = base_info
        
        
        return  pd.DataFrame(ships_info)
    
    
    def get_shipyard_info(self):
        """ Returns the info about shipyards in all of the board. """
        shipyards_info = {}
        
        for shipyard_id, shipyard in self.board.shipyards.items():
            base_info = {
                "my_shipyard": 0, 
                "player_halite": 0, "moves": 0, "position": None
            }
            
            base_info['player_halite'] = shipyard.player.halite
            base_info['moves'] = count_moves(self.ship.position, shipyard.position)
            base_info['position'] = (shipyard.position.x, shipyard.position.y)
            
            if shipyard_id in self.board.current_player.shipyard_ids:
                base_info['my_shipyard'] = 1
                
                shipyards_info[shipyard_id] = base_info
            else:
                shipyards_info[shipyard_id] = base_info
        
        return pd.DataFrame(shipyards_info)
        
        
    def generate_grid_df(self):
        """ Generates a Dataframe describing the information of objects and cells in the 5x5 grid of the ship. """
        all_dirs = {}
        
        for direction, cell in self.grid.items():
            
            base_info = {
                "ship_id": None, "shipyard_id": None, 
                "my_ship": 0, "my_shipyard": 0,
                "halite": 0, "moves": 0
            }
            
            if cell.ship != None: 
                base_info["ship_id"] = cell.ship.id
                if cell.ship.id in self.ship.player.ship_ids:
                    base_info["my_ship"] = 1
    
            if cell.shipyard != None: 
                base_info["shipyard_id"] = cell.shipyard.id
                if cell.shipyard.id in self.ship.player.shipyard_ids:
                    base_info['my_shipyard'] = 1
                    
            base_info['halite'] = cell.halite
            # The number of letters in the direction would indicate the number of moves needed to get there
            base_info['moves'] = len(direction)
            
            all_dirs[direction] = base_info
            
        return pd.DataFrame(all_dirs)
      

In [140]:
####################
# Helper Functions #
####################

def grid_5(cell):
    """
        Returns a dictionary based on the cells in 
        the surrounding 5x5 area of a given cell
    """
    # Main ones
    north, south, west, east = cell.north, cell.south, cell.west, cell.east

    # Secondary ones
    nn, ss, ww,ee = north.north, south.south, west.west, east.east
    
    # The length of the key corresponds to the number of moves needed to get to the cell
    return {
        'N': north, 'S': south, 'W': west, 'E': east, 'NW': north.west, 'NE': north.east, 
        'SW': south.west, 'SE': south.east, 'WW': ww, 'EE': ee, 'NN': nn, 'SS': ss ,
        'NEN': nn.east, 'NWN': nn.west, 'SES': ss.east, 'SWS': ss.west,  'SEE': ee.south, 
        'NEE': ee.north, 'SWW': ww.south, 'NWW': ww.north, 'SEES': ee.south.south , 
        'NEEN': ee.north.north, 'NWWN': ww.north.north, 'SWWS': ww.south.south
    }  
def count_moves(point1, point2, size=21):
    """ 
        Returns the minimum number of between moves 
        to go from point1 to point2.
    """
    # Break the points into coordinates
    x1, y1 = point1.x, point1.y
    x2, y2 = point2.x, point2.y
    
    # For both x and y they are two type of paths to take
    diff_x_1 = abs(x2 - x1) 
    diff_x_2 = abs(size - x2 + x1)
    diff_y_1 = abs(y2 - y1)
    diff_y_2 = abs(size - y2 + y1)
    
    opt1 = diff_x_1 + diff_y_1
    opt2 = diff_x_1 + diff_y_2
    opt3 = diff_x_2 + diff_y_1
    opt4 = diff_x_2 + diff_y_2
    
    return min(opt1, opt2, opt3, opt4)


# Global values
acts = {
    "N": ShipAction.NORTH, 'S': ShipAction.SOUTH,
    'W': ShipAction.WEST , 'E' : ShipAction.EAST,
    'spawn': ShipyardAction.SPAWN, 'convert': ShipAction.CONVERT,
    'mine': None
}

def log(text, step=1):
    if step == 0:
        with open("log.txt", "w") as text_file:
            text = str(text) + '\n'
            text_file.write(text)
    else:
        with open("log.txt", "a") as text_file:
            text = str(text) + '\n'
            text_file.write(text)

In [2]:
from kaggle_environments import make
from kaggle_environments.envs.halite.helpers import *

# Create a test environment for use later
board_size = 21
environment = make("halite", configuration={"size": board_size, "startingHalite": 5000}, debug=True)
agent_count = 4
environment.reset(agent_count)
state = environment.state[0]

In [153]:
# Change the behviour of my own movement from board.current_player
board = Board(state.observation, environment.configuration)
board.current_player.ships[0].next_action = ShipAction.CONVERT
board.ships['0-2'].next_action = ShipAction.CONVERT
board.ships['0-3'].next_action = ShipAction.CONVERT
board.ships['0-4'].next_action = ShipAction.CONVERT
board = board.next()
board.current_player.shipyards[0].next_action = ShipyardAction.SPAWN
board.shipyards['1-2'].next_action = ShipyardAction.SPAWN
board.shipyards['1-3'].next_action = ShipyardAction.SPAWN
board.shipyards['1-4'].next_action = ShipyardAction.SPAWN
board = board.next()

# Current-Player
board.current_player.ships[0].next_action  = ShipAction.EAST
board.current_player.shipyards[0].next_action = ShipyardAction.SPAWN
# Opponents
board.ships['2-2'].next_action = ShipAction.NORTH
board.shipyards['1-2'].next_action = ShipyardAction.SPAWN
board.ships['2-3'].next_action = ShipAction.WEST
board.shipyards['1-3'].next_action = ShipyardAction.SPAWN
board.ships['2-4'].next_action = ShipAction.EAST
board.shipyards['1-4'].next_action = ShipyardAction.SPAWN
board = board.next()

# Current-Player
board.current_player.ships[0].next_action  = ShipAction.NORTH
board.current_player.ships[1].next_action  = ShipAction.SOUTH
board.current_player.shipyards[0].next_action = ShipyardAction.SPAWN
# Opponents
board.ships['2-2'].next_action = ShipAction.WEST
board.shipyards['1-2'].next_action = ShipyardAction.SPAWN
board.ships['2-3'].next_action = ShipAction.NORTH
board.shipyards['1-3'].next_action = ShipyardAction.SPAWN
board.ships['2-4'].next_action = ShipAction.SOUTH
board.shipyards['1-4'].next_action = ShipyardAction.SPAWN
board = board.next()

# Current-Player
board.current_player.ships[0].next_action  = ShipAction.EAST
board.current_player.ships[1].next_action  = ShipAction.SOUTH
board.current_player.ships[2].next_action  = ShipAction.EAST
# Opponents
board.ships['2-2'].next_action = ShipAction.WEST
board.ships['2-3'].next_action = ShipAction.EAST
board.ships['2-4'].next_action = ShipAction.SOUTH
board = board.next()



ship = board.current_player.ships[0]
loc = ShipLocation(board, ship)

In [136]:
board.ships

{'2-1': <kaggle_environments.envs.halite.helpers.Ship at 0x7feb5a9cf5e0>,
 '3-1': <kaggle_environments.envs.halite.helpers.Ship at 0x7feb5a9cf430>,
 '4-1': <kaggle_environments.envs.halite.helpers.Ship at 0x7feb5a9cf880>,
 '2-2': <kaggle_environments.envs.halite.helpers.Ship at 0x7feb5a9dbac0>,
 '2-3': <kaggle_environments.envs.halite.helpers.Ship at 0x7feb5b2b2d00>,
 '2-4': <kaggle_environments.envs.halite.helpers.Ship at 0x7feb5b2b25e0>}

In [33]:
board.shipyards

{'1-1': <kaggle_environments.envs.halite.helpers.Shipyard at 0x7feb5b0335b0>,
 '1-2': <kaggle_environments.envs.halite.helpers.Shipyard at 0x7feb5b0333a0>,
 '1-3': <kaggle_environments.envs.halite.helpers.Shipyard at 0x7feb5b033280>,
 '1-4': <kaggle_environments.envs.halite.helpers.Shipyard at 0x7feb5b033160>}

In [139]:
print(board)

| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 1 | 0 |a0 | 0 | 0 | 0 | 0 | 0 |b0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0A|a0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0B| 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 |a0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 

In [34]:
board.current_player.ships

In [121]:
board.current_player.shipyards

In [122]:
board.shipyards

{'1-1': <kaggle_environments.envs.halite.helpers.Shipyard at 0x7feb5aacfaf0>,
 '1-2': <kaggle_environments.envs.halite.helpers.Shipyard at 0x7feb5aacf820>,
 '1-3': <kaggle_environments.envs.halite.helpers.Shipyard at 0x7feb5aacf8b0>,
 '1-4': <kaggle_environments.envs.halite.helpers.Shipyard at 0x7feb5aacf430>}

In [131]:
yard = board.shipyards['1-4']
l = ShipLocation(board, yard).generate_grid_df().T

In [132]:
l

,ship_id,shipyard_id,my_ship,my_shipyard,halite,moves
N,NaN,NaN,0,0,5.52,1
S,NaN,NaN,0,0,62.933,1
W,NaN,NaN,0,0,8.832,1
E,NaN,NaN,0,0,42.216,1
NW,NaN,NaN,0,0,0,2
NE,NaN,NaN,0,0,0,2
SW,NaN,NaN,0,0,0,2
SE,NaN,NaN,0,0,5.412,2
WW,NaN,NaN,0,0,19.874,2
EE,NaN,NaN,0,0,0,2


In [154]:
z = loc.get_shipyard_info()

In [158]:
z.T['position'][0][0]

5

In [119]:
z.T['moves'].idxmin()

'1-1'

In [88]:
loc.generate_grid_df()

,N,S,W,E,NW,NE,SW,SE,WW,EE,...,SES,SWS,SEE,NEE,SWW,NWW,SEES,NEEN,NWWN,SWWS
ship_id,NaN,NaN,NaN,NaN,NaN,NaN,4-1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
shipyard_id,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1-1,NaN,NaN,NaN,NaN,NaN
my_ship,0.000,0.000,0.0,0.00,0.0,0.0,1,0.0,0.000,0.0,...,0.000,0.0,0.000,0.0,0,0.0,0.0,0.0,0.0,0.000
my_shipyard,0.000,0.000,0.0,0.00,0.0,0.0,0,0.0,0.000,0.0,...,0.000,0.0,0.000,0.0,1,0.0,0.0,0.0,0.0,0.000
halite,28.706,19.874,0.0,11.04,0.0,0.0,8.489,0.0,62.933,0.0,...,8.832,0.0,4.417,0.0,0,0.0,0.0,0.0,0.0,5.412
moves,1.000,1.000,1.0,1.00,2.0,2.0,2,2.0,2.000,2.0,...,3.000,3.0,3.000,3.0,3,3.0,4.0,4.0,4.0,4.000


In [144]:
d = loc.get_ship_info()

In [145]:
d

,3-1,4-1,2-2,2-3,2-4
my_ship,1.000,1.000,0.000,0.000,0.0
cargo,29.226,8.489,11.906,5.412,0.0
moves,5.000,2.000,6.000,12.000,22.0


In [95]:
class Decision_Ship:
    """ 
        Decides ship's next move:

        params: 
            board = the board that we will base our decisions on
            ship = the ship we are deciding for
            step = the steps into the stimulation
        
        returns:
            determine: returns the next-action that should be taken
    """
    # Implement a function that measures the nearest shipyards
    def __init__(self, board, ship):
        # Given values
        self.board = board
        self.step = board.observation['step']
        self.ship = ship
        self.ship_cargo = ship.halite
        self.current_cell = ship.cell
        
        # Some usefull properties
        self.player = self.board.current_player  # Player
        self.simple_moves = {'N': ship.cell.north, 'S': ship.cell.south, 'W': ship.cell.west, 'E': ship.cell.east}
        
        # All moves ship can take
        self.moves = {"N": ShipAction.NORTH, 'S': ShipAction.SOUTH, 'W': ShipAction.WEST , 
                      'E' : ShipAction.EAST, 'convert': ShipAction.CONVERT, 'mine': None}
    
        # Weights of different moves
        self.weights = {"N": 0, "E": 0, "W": 0, "S": 0, "mine": 0, "convert": 0}
        
        # The object's relative situation to other ship/shipyards
        self.locator = ShipLocation(board, ship)
        self.Ships = self.locator.get_ship_info()
        self.Shipyards = self.locator.get_shipyard_info()
        self.grid = self.locator.generate_grid_df()
        
        # Default move which is set to mining (None)
        self.next_move = None
        
        
        
    def determine(self):
        """ Returns the next action decided for the ship based on the observations that have been made. """
        # Weight different moves:
        
        # Get the weights for main four directions
        self.weight_moves()
        
        # Decide between moves
        sorted_weights = {k: v for k, v in sorted(self.weights.items(), key=lambda item: item[1], reverse=True)}
        
        log('  weights:' + str(sorted_weights))
        
        # Choose the action with highest value if it has not been eliminated
        for action, weight in sorted_weights.items():
            if action in self.moves.keys():
                return self.moves[action]
        
        # If none were chosen, then just return the default move which is mining
        return self.next_move
    
    """
        The should be a base function which checks on shipyards and 
        evaludates if they need protection or not.
    """
    
    def weight_moves(self):
        """ This function sets the self.weight parameters
        NOTE: Weights should be between -10 to 10. """
        dirs = list(self.grid.columns)
        # Iterate through different directions
        for direction in dirs:
            wdir = 0
            # If there was a ship
            if not pd.isna(self.grid[direction].ship_id):
                # If it was my ship
                if self.grid[direction].my_ship == 1:
                    wdir += self.avoid_self_colision(self.grid[direction].ship_id)
                else: 
                    # If it was not my ship 
                    self.deal_enemy_ship(self.grid[direction].ship_id)
    
    
    
    def deal_enemy_ship(self, ship_id):
        """ This function is to decide wether I should attack the enemy ship or run away from it. """
        oppCargo = self.Ships[ship_id].cargo
        moves = self.Ships[ship_id].moves
        w = 0
        
        if self.ship_cargo > oppCargo:
            # If I had more halite then should get away
            # If the ship was close enough then move
            if moves == 1:
                self.weights['mine'] = -8 * (self.ship_cargo + 2)
                self.weight['convert'] = -5 * (self.ship_cargo + 3)
            
            w = -10 * self.ship_cargo
            
            # Implement: find the directions to the nearest shipyard and add the values to the corresponding weight move
            
        else:
            # If the ship had more halite then attack, it should take into acount
            # how far is it from the nearst shipayrd and also how many ships have 
            # surounded that shipyard
            closest_shipyard_id = self.Shipyards.T['moves'].idxmin()
            closest_shipyard_dist = self.Shipyards[closest_shipyard_id].moves
            
            w += 10 * (oppCargo - self.ship_cargo) / closest_shipyard_dist
            
        return w
                
    
    # Implement this function
    def analyze_shipyard_souroundings(self, shipyard_id):
        """ Checks to see if a given shipyard needs protection or not? """
        shipyard = self.Shipyards[shipyard_id]
        
        shipyard_grid = ShipLocation(board, shipyard).generate_grid_df()         
            
    
    def avoid_self_colision(self, ship_id):
        """ This function is called to come up with a weight to avoid collision 
        with any of my own ships. """
        # The amount of cargo in the ship would effect the decision
        w = (self.Ships[ship_id].cargo + 5)
        # This would also encourage for mining
        self.weights['mine'] += 10
        # -10 is the highest negative weight given
        return w * -10
        
                    
    
    def near_end(self):
        """ Returns True if the game is almost over. """
        count = 0
        
        # If the halite was less than 500 and it had no ships
        for opp in self.board.opponents:
            if opp.halite < 500 and len(opp.ships) == 0 and self.player.halite > opp.halite: count += 1
            if opp.halite > 3000 and len(opp.ships) > 1: count -= 1
                
        # If count was more than 2 return True
        return count >= 2

In [120]:
class ShipyardDecesion:
    def __init__(self, board, shipyard):
        self.board = board
        self.shipyard = shipyard

In [106]:
from kaggle_environments import make
from kaggle_environments.envs.halite.helpers import *

acts = {
    "N": ShipAction.NORTH, 'S': ShipAction.SOUTH,
    'W': ShipAction.WEST , 'E' : ShipAction.EAST,
    'spawn': ShipyardAction.SPAWN, 'convert': ShipAction.CONVERT,
    'mine': None
}

# Create a test environment for use later
board_size = 21
environment = make("halite", configuration={"size": board_size, "startingHalite": 5000}, debug=True)
agent_count = 4
environment.reset(agent_count)
state = environment.state[0]
board = Board(state.observation, environment.configuration)

for i in range(12):
    player = board.current_player
    print(board.observation['step'], '#####################################')
    
    for ship in player.ships:
        log(str(ship.position) + ', id=' + ship.id)
        decider = Decision_Ship(board, ship)
        ship.next_action = decider.determine()
        print('----------------------')
    for shipyard in player.shipyards:
        if player.halite > 2000:
            shipyard.next_action = acts['spawn']
                
    print(player.next_actions)
    
    board = board.next()

0 #####################################
----------------------
{'0-1': 'CONVERT'}
1 #####################################
{'1-1': 'SPAWN'}
2 #####################################
----------------------
{'2-1': 'SOUTH', '1-1': 'SPAWN'}
3 #####################################
----------------------
----------------------
{'2-1': 'WEST', '3-1': 'WEST', '1-1': 'SPAWN'}
4 #####################################
----------------------
----------------------
----------------------
{'2-1': 'EAST', '4-1': 'SOUTH', '1-1': 'SPAWN'}
5 #####################################
----------------------
----------------------
{'3-1': 'SOUTH', '5-1': 'SOUTH', '1-1': 'SPAWN'}
6 #####################################
----------------------
----------------------
----------------------
{'6-1': 'WEST'}
7 #####################################
----------------------
----------------------
----------------------
{}
8 #####################################
----------------------
----------------------
-----------------

# Actual writting zone:

In [102]:
%%writefile current.py
from kaggle_environments.envs.halite.helpers import *
import pandas as pd

class Decision_Ship:
    """ 
        Decides ship's next move:

        params: 
            board = the board that we will base our decisions on
            ship = the ship we are deciding for
            step = the steps into the stimulation
        
        returns:
            determine: returns the next-action that should be taken
    """
    # Implement a function that measures the nearest shipyards
    def __init__(self, board, ship):
        # Given values
        self.board = board
        self.step = board.observation['step']
        self.ship = ship
        # Some usefull properties
        self.ship_halite = ship.cell.halite # Ship's halite
        self.player = self.board.current_player  # Player
        # All moves ship can take
        self.moves = {"N": ShipAction.NORTH, 'S': ShipAction.SOUTH, 'W': ShipAction.WEST , 
                      'E' : ShipAction.EAST, 'convert': ShipAction.CONVERT, 'mine': None}
        
        # 5x5 grid around the ship's cell
        self.grid = grid_5(self.ship.cell)
        # Weights of different moves
        self.weights = {"N": 0, "E": 0, "W": 0, "S": 0, "mine": 0, "convert": 0}
        
        self.next_move = None
        
    def determine(self):
        """ Return the next action """
        # Weight different moves:
        
        # First stage: eliminations
        self.first_stage()
        # Get the weights for main four directions
        self.weight_moves()
        # Get the mining weight
        self.weight_mining()
        # get the converting weight
        self.weight_convert()
        
        # Decide between moves
        sorted_weights = {k: v for k, v in sorted(self.weights.items(), key=lambda item: item[1], reverse=True)}
        
        log('  weights:' + str(sorted_weights))
        
        # Choose the action with highest value if it has not been eliminated
        for action, weight in sorted_weights.items():
            if action in self.moves.keys():
                return self.moves[action]
        
        return self.next_move

    
    def weight_mining(self):
        """ Weights mining move for the ship. """
        # Direct correlation with the amount of halite in current_cell - exponential
        # Indirect-corr with the number of steps at the beginning of the game should move around
        self.weights["mine"] = round(self.ship.cell.halite ** (0.3 + (self.step // 100)), 3)


    def weight_convert(self, threshold=2000):
        """ Weights converting for the ship. """
        # Implement: a function that decides if this the best ship to convert to the shipyard
        # Some things to take into account
        # 1. If they are no shipyards left
        no_yards_left = len(self.player.shipyards) == 0
        # 2. If it is the end of the game and we have more than 500 halite in our cargo
        end_of_game_conversion = (self.step > 395 or self.near_end()) and self.ship.halite >= 500
        # 3. There will be a threshold for the amount of cargo any ship could have
        threshhold_reach = self.ship.halite > threshold
        # 4. On shipyard already
        on_shipyard = self.ship.cell.shipyard != None
        
        if (no_yards_left or end_of_game_conversion or threshhold_reach) and not on_shipyard: 
            self.weights['convert'] = 100
        elif not on_shipyard:
            self.weights['convert'] = round((self.ship.halite - 1000) / (self.step // 50 + 1) , 3)
        else:
            self.weights['convert'] = round(2 * (self.ship.halite - 1000) / (self.step // 50 + 1), 3)
    
    
    def weight_moves(self):
        """ This function sets the self.weight parameter for N, W, S, and E. """
        for Dir, cell in self.grid.items():
            # N, W, E, AND S:
            if Dir in self.moves.keys() and len(Dir) == 1: 
                # Instantiate the weight for the Direction
                self.weights[Dir] = self.weight_cell(cell)
                
                # Go through all other ones
                for sub_Dir, sub_cell in self.grid.items():
                    if Dir in sub_Dir and Dir != sub_Dir and len(sub_Dir) == 1:
                        self.weights[Dir] += self.weight_cell(sub_cell) * (5 - len(Dir))

                    
    
    def first_stage(self):
        """
            Eliminate some moves before weighting the moves
            'DONT_GO': avoid that direction
            'GET_away': Don't stay in the current position and took one of the other path than this one
        """
        # If there was a ship/shipyard in E,N,W,or E
        for Dir, cell in self.grid.items():
            if len(Dir) == 1:
                cell_ship = cell.ship
                cell_yard = cell.shipyard

                # If there is a ship:
                if cell_ship != None:
                    # If it is one of my ships
                    if cell_ship.id in self.player.ship_ids:
                        # 'DONT_GO'
                        del self.moves[Dir]
                    else:
                        myCargo = self.ship.halite
                        oppCargo = cell_ship.halite
                        # If I had more cargo then get_away
                        if oppCargo < myCargo:
                            # 'GET_AWAY'
                            del self.moves[Dir]
                            # To avoid errors first check to see if the value is there or not
                            if 'mine' in self.moves.keys():
                                del self.moves['mine'] 
                            if 'convert' in self.moves.keys():
                                del self.moves['convert']
    
    
    def weight_cell(self, cell):
        """ Weights a cell only based on its properties and relative halite. """
        # 1. The difference of cell's halite with the current cell's halite
        # 2. If there was a ship then it would be related to the amount of difference in cargo and 1/step
        # 3. If there was my shipyard then it would have a direct corr with the cargo and undirect corr with the steps of the game 
        # 4. If there was an enemy shipyard then + with player's halite, - with # of yards of enemt, - with ship's cargo
        w, cell_ship, cell_shipyard = 0, cell.ship, cell.shipyard
        
        # Mine
        w += (cell.halite - self.ship_halite) + 2
        if cell_ship != None:
            if cell_ship.id in self.player.ship_ids:
                w += -10 * (cell_ship.halite + self.ship.halite)
            else:
                myCargo = self.ship.halite
                oppCargo = cell_ship.halite
                # defensive                
                w += (oppCargo - myCargo) * 3
                
        if cell_shipyard != None:
            if cell_shipyard.id in self.player.shipyard_ids:
                # Defensive
                w += (self.ship.halite + 20) * 3
            else:
                oppYards = len(cell_shipyard.player.shipyards)
                oppHalite = cell_shipyard.player.halite
                # offensive
                w += (oppHalite + 10) / (oppYards + 0.2) 
                
        return round(w, 3)
    
    
    def near_end(self):
        """ Returns True if the game is almost over. """
        count = 0
        # If the halite was less than 500 and it had no ships
        for opp in self.board.opponents:
            if opp.halite < 500 and len(opp.ships) == 0 and self.player.halite > opp.halite: count += 1
        # If count was more than 2 return True
        return count >= 2

    
class ShipLocation:
    def __init__(self, board, ship, grid):
        self.board = ship
        self.ship = ship
        # Get the grid
        self.grid = grid
        
        
    def other_objects(self):
        """ Returns two Dataframes about the information of ships and shipyards on the map. """
        ships_info = {}
        for ship in self.board.ships:
            base_info = {
                "my_ship": False, 
                "halite": 0, "moves": 0
            }
            
            base_info['halite'] = ship.cell.halite
            base_info['moves'] = count_moves(self.ship.position, ship.position)
            
            if ship.id in self.ship.player.ship_ids and ship.id != self.ship.id:
                base_info['my_ship'] = True
                
                ships_info[ship.id] = base_info
            elif ship.id != self.ship.id:
                ships_info[ship.id] = base_info
        
        shipyards_info = {}
        for shipyard in self.board.shipyards:
            base_info = {
                "my_shipyard": False, 
                "halite": 0, "moves": 0
            }
            
            base_info['halite'] = ship.cell.halite
            base_info['moves'] = count_moves(self.ship.position, ship.position)
            
            if shipyard.id in self.ship.player.shipyard_ids:
                base_info['my_shipyard'] = True
                
                shipyards_info[shipyard.id] = base_info
            else:
                shipyards_info[shipyard.id] = base_info
            
        return pd.DataFrame(shipyards_info), pd.DataFrame(ships_info)
        
        
    def generate_grid_df(self):
        """ Generates a Dataframe describing the information of objects and cells in the 5x5 grid of the ship. """
        all_dirs = {}
        
        for direction, cell in self.grid.items():
            
            base_info = {
                "ship_id": None, "shipyard_id": None, 
                "my_ship": False, "my_shipyard": False,
                "halite": 0, "moves": 0
            }
            
            if cell.ship != None: 
                base_info["ship_id"] = cell.ship.id
                if cell.ship.id in self.ship.player.ship_ids:
                    base_info["my_ship"] = True
    
            if cell.shipyard != None: 
                base_info["shipyard_id"] = True
                if cell.shipyard.id in self.ship.player.shipyards_ids:
                    base_info['my_shipyard'] = True
                    
            base_info['halite'] = cell.halite
            # The number of letters in the direction would indicate the number of moves needed to get there
            base_info['moves'] = len(direction)
            
            all_dirs[direcction] = base_info
            
        return pd.DataFrame(all_dirs)
        
####################
# Helper Functions #
####################

def grid_5(cell):
    """
        Returns a dictionary based on the cells in 
        the surrounding 5x5 area of a given cell
    """
    # Main ones
    north, south, west, east = cell.north, cell.south, cell.west, cell.east

    # Secondary ones
    nn, ss, ww,ee = north.north, south.south, west.west, east.east
    
    # The length of the key corresponds to the number of moves needed to get to the cell
    return {
        'N': north, 'S': south, 'W': west, 'E': east, 'NW': north.west, 'NE': north.east, 
        'SW': south.west, 'SE': south.east, 'WW': ww, 'EE': ee, 'NN': nn, 'SS': ss ,
        'NEN': nn.east, 'NWN': nn.west, 'SES': ss.east, 'SWS': ss.west,  'SEE': ee.south, 
        'NEE': ee.north, 'SWW': ww.south, 'NWW': ww.north, 'SEES': ee.south.south , 
        'NEEN': ee.north.north, 'NWWN': ww.north.north, 'SWWS': ww.south.south
    }        


def count_moves(point1, point2, size=21):
    """ 
        Returns the minimum number of between moves 
        to go from point1 to point2.
    """
    # Break the points into coordinates
    x1, y1 = point1.x, point1.y
    x2, y2 = point2.x, point2.y
    
    # For both x and y they are two type of paths to take
    diff_x_1 = abs(x2 - x1) 
    diff_x_2 = abs(size + x2 - x1)
    diff_y_1 = abs(y2 - y1)
    diff_y_2 = abs(size + y2 - y1)
    
    opt1 = diff_x_1 + diff_y_1
    opt2 = diff_x_1 + diff_x_2
    opt3 = diff_x_2 + diff_y_1
    opt4 = diff_x_2 + diff_x_2
    
    return min(opt1, opt2, opt3, opt4)


def log(text, step=1):
    if step == 0:
        with open("log.txt", "w") as text_file:
            text = str(text) + '\n'
            text_file.write(text)
    else:
        with open("log.txt", "a") as text_file:
            text = str(text) + '\n'
            text_file.write(text)


# Global values
acts = {
    "N": ShipAction.NORTH, 'S': ShipAction.SOUTH,
    'W': ShipAction.WEST , 'E' : ShipAction.EAST,
    'spawn': ShipyardAction.SPAWN, 'convert': ShipAction.CONVERT,
    'mine': None
}

log('logs:', 0)

def agent(obs, config):
    # Make the board
    board = Board(obs,config)
    #Step of the board
    step = board.observation['step']
    # Current player info
    me = board.current_player # Player Object
    
    new_board = Board(obs,config)
    log(step)
    for ship in me.ships:
        log('ship-id:' + ship.id + ', pos:' + str(ship.position))
        decider = Decision_Ship(new_board, new_board.ships[ship.id])
        ship.next_action = decider.determine()
        
        new_board = board.next()
    
    for shipyard in me.shipyards:
        # If there were no ships on the yard
        if new_board.shipyards[shipyard.id].cell.ship == None and step < 392:
            if len(me.ships) == 0:
                shipyard.next_action = acts['spawn']

            if step % 3 == 1:
                shipyard.next_action = acts['spawn']

            if step > 200 and me.halite > 10000 + len(me.ships) * 1000:
                shipyard.next_action = acts['spawn']
        
        new_board = board.next()
        
    return me.next_actions

Overwriting current.py


In [103]:
from kaggle_environments import make
from kaggle_environments.envs.halite.helpers import *

env = make("halite", debug=True)
env.run(["random", "current.py", 'random', 'attacker_c_2.py'])
env.render(mode="ipython", width=800, height=600)